In [67]:
class Phylogeny_Data():
##This class hold information and lists of strain objects
##To represent the strains we are using in our phylogeny

    names = []
    
    ##Holds an aligned sequence file
    alignment_file = ""
    
    strains = []
    
    def __init__(self):
    ##Creates a Phylogeny_Data obj
        alignment_file = ""
        names = []
    
    def get_names(self):
    ##Returns a list of names based on the current alignment_file
    
        names = []
    
        file = open(self.alignment_file)
        
        for line in file:

            ##Split up into list of words
            words = line.split()
            
            if len(words) == 0:
                self.names = names
                file.close()
                return names
            else:
                ##take first word
                name = words[0]
                ##add to list
                names.append(name)
                
    def get_strains(self):
    ##Returns a list of strain objects from our alignment data
      
        for self.name in self.names:

            genome = Strain()
            genome.name = self.name
            
            file = open(self.alignment_file)
        
            for line in file:
                ##Split up into list of words
                words = line.split()
            
                if len(words) == 0:
                    continue
                elif words[0] == self.name:
                    genome.sequences.append(words[1])
                elif words[0] == "end":
                    break
                    
            self.strains.append(genome)
    
        file.close()
        return self.strains
            
class Strain():
##This class hold info about the strain we are analyzing

    sequences = []
    ##A list to hold sequences

    name = ""
    ##String to hold strain name

    def __init__(dictionary, seq_name):
    ##Constructs a strain obj using the given parameters
        sequences = dictionary
        name = seq_name
        
    def __init__(self):
    ##No args constructor
        name = ""
        sequences = None
        
    def to_string(self):
    ## returns a string containing obj info
    
        ## name line 
        string = ">" + str(self.name) + "\n"
        
        for self.seq in self.sequences:
            string += str(self.seq)
            
        return string
    
phylogeny = Phylogeny_Data()

phylogeny.alignment_file = "Alignments.txt"

print(str(phylogeny.get_names()))

curr_strains = phylogeny.get_strains() 

output = open("Output.txt", 'w')


['NC_048212.1', 'NC_010646.1', 'NC_014470.1', 'NC_016994.1', 'NC_016995.1', 'NC_016996.1', 'NC_026011.1', 'NC_046954.1', 'NC_017083.1', 'NC_003045.1', 'NC_006213.1', 'NC_046965.1', 'NC_048214.1', 'NC_010800.1', 'NC_038294.1', 'NC_019843.3', 'NC_011549.1', 'NC_016993.1', 'NC_011550.1', 'NC_016991.1', 'NC_016992.1', 'NC_039208.1', 'NC_046955.1', 'NC_032730.1', 'NC_028806.1', 'NC_002306.3', 'NC_023760.1', 'NC_030292.1', 'Alphacoronavirus', 'NC_028752.1']
